In [101]:
from sympy import *

4

In [450]:
n,m = 3,2
n,m = symbols("n m")
grads = {"xx":(n,n),"uu":(m,m),"ux":(m,n),"xu":(n,m),"x":(1,n),"u":(1,m)}
x1,u1,xm,um,x2,u2,x1d,t = symbols('x1 u1 xm um x2 u2 xdot_1 t')
L = {wrt+num:MatrixSymbol("L"+wrt+"_"+num,*sze) for (wrt,sze) in grads.items() for num in ["1","m","2"]}
A = MatrixSymbol("A",n,n)
B = MatrixSymbol("B",n,m)
C = MatrixSymbol("C",n,m)
In = Identity(n)
Im = Identity(m)
Zn = ZeroMatrix(n,n)
Zm = ZeroMatrix(m,m)
Za = ZeroMatrix(n,n)
Zb = ZeroMatrix(n,m)

x1 = MatrixSymbol("x_1",n,1)
u1 = MatrixSymbol("u_1",m,1)
xm = MatrixSymbol("x_m",n,1)
um = MatrixSymbol("u_m",m,1)
x2 = MatrixSymbol("x_2",n,1)
u2 = MatrixSymbol("u_2",m,1)
x1m2 = BlockMatrix([[x1.T,u1.T,xm.T,um.T,x2.T,u2.T]]).T
x12 = BlockMatrix([[x1.T,u1.T,x2.T,u2.T]]).T
x12.shape

(2*m + 2*n, 1)

In [451]:
In/2

(1/2)*I

In [452]:
# Find a,b,c for quadratic polynomial
x_abc = Matrix([[x1,x1d,x2]]).T
p = lambda t: Matrix([[t**2,t,1]])
# T = BlockMatrix([[Za,Za,In],[Za,In,Za],[In*t**2,t*In,1*In]])
T = Matrix([[0,0,1],[0,1,0],p(t)])
abc = T.inv()*x_abc
pprint(simplify(abc))

⎡         ⎛-1       1    ⎞⎤
⎢-ẋ₁ + t⋅⎜───⋅x₁ + ──⋅x₂⎟⎥
⎢         ⎜  2       2   ⎟⎥
⎢         ⎝ t       t    ⎠⎥
⎢─────────────────────────⎥
⎢            t            ⎥
⎢                         ⎥
⎢         ẋ₁ + 0         ⎥
⎢                         ⎥
⎣           x₁            ⎦


In [453]:
# Find M matrix
N = p(t/2)*T.inv()
pprint(N)
N = BlockMatrix([[In*N[0],In*N[1],In*N[2]]])

⎡     t     ⎤
⎢3/4  ─  1/4⎥
⎣     4     ⎦


In [454]:
# Matrix to convert from x12 to x1d2
x12 = Matrix([[x1,u1,x2,u2]]).T
x1m2 = Matrix([[x1,u1,xm,um,x2,u2]]).T
F = BlockMatrix([[I_n,Zb,Za,Zb],
                 [A,B,Za,C],
                 [Za,Zb,I_n,Zb]])
pprint(F)
M2 = block_collapse(N*F)
pprint(M2)

⎡I  0  0  0⎤
⎢          ⎥
⎢A  B  0  C⎥
⎢          ⎥
⎣0  0  I  0⎦
⎡        t    t           t  ⎤
⎢3/4⋅I + ─⋅A  ─⋅B  1/4⋅I  ─⋅C⎥
⎣        4    4           4  ⎦


In [455]:
# Matrix to convert from x12 to x1m2
# E = BlockMatrix([[In,Zb,Za,Zb],
#                  [Zb.T,Im,Zb.T,Im],
#                  M2,
#                  [Zb.T,Im/2,Zb.T,Im/2],
#                  [Zn,Zb,In,Zb],
#                  [Zb.T,Zm,Zb.T,Im]])
M1 = MatrixSymbol("M_1",n,n)
M2 = MatrixSymbol("M_2",n,m)
M3 = In/4
M4 = MatrixSymbol("M_4",n,m)

E = BlockMatrix([[In,Zb,Za,Zb],
                 [Zb.T,Im,Zb.T,Zm],
                 [M1,M2,M3,M4],
                 [Zb.T,Im/2,Zb.T,Im/2],
                 [Za,Zb,In,Zb],
                 [Zb.T,Zm,Zb.T,Im]])

pprint(E)
E.shape

⎡I     0      0      0  ⎤
⎢                       ⎥
⎢0     I      0      0  ⎥
⎢                       ⎥
⎢M₁   M₂    1/4⋅I   M₄  ⎥
⎢                       ⎥
⎢0   1/2⋅I    0    1/2⋅I⎥
⎢                       ⎥
⎢0     0      I      0  ⎥
⎢                       ⎥
⎣0     0      0      I  ⎦


(3*m + 3*n, 2*m + 2*n)

In [456]:
# Stage Cost
L_Hessian = BlockMatrix([
    [L["xx1"],L["xu1"],Za,Zb,Za,Zb],
    [L["ux1"],L["uu1"],Zb.T,Zm,Zb.T,Zm],
    [Za,Zb,4*L["xxm"],4*L["xum"],Za,Zb],
    [Zb.T,Zm,4*L["uxm"],4*L["uum"],Zb.T,Zm],
    [Za,Zb,Za,Zb,L["xx2"],L["xu2"]],
    [Zb.T,Zm,Zb.T,Zm,L["ux2"],L["uu2"]]])

pprint(L_Hessian)
L_Hessian.shape

⎡Lxx₁  Lxu₁    0       0      0     0  ⎤
⎢                                      ⎥
⎢Lux₁  Luu₁    0       0      0     0  ⎥
⎢                                      ⎥
⎢ 0     0    4⋅Lxxₘ  4⋅Lxuₘ   0     0  ⎥
⎢                                      ⎥
⎢ 0     0    4⋅Luxₘ  4⋅Luuₘ   0     0  ⎥
⎢                                      ⎥
⎢ 0     0      0       0     Lxx₂  Lxu₂⎥
⎢                                      ⎥
⎣ 0     0      0       0     Lux₂  Luu₂⎦


(3*m + 3*n, 3*m + 3*n)

In [457]:
L_Grad = BlockMatrix([[L["x1"],L["u1"],L["xm"],L["um"],L["x2"],L["u2"]]])
pprint(L_Grad)
L_Grad.shape

[Lx₁  Lu₁  Lxₘ  Luₘ  Lx₂  Lu₂]


(1, 3*m + 3*n)

In [458]:
# Find Stage cost in terms of x12
L_H = block_collapse(E.T*L_Hessian*E)
pprint(L_H.blocks[:,3])

⎡                    T            T                         ⎤
⎢                2⋅M₁ ⋅Lxuₘ + 4⋅M₁ ⋅Lxxₘ⋅M₄                 ⎥
⎢                                                           ⎥
⎢       ⎛             T     ⎞   ⎛             T     ⎞       ⎥
⎢   1/2⋅⎝2⋅Luuₘ + 4⋅M₂ ⋅Lxuₘ⎠ + ⎝2⋅Luxₘ + 4⋅M₂ ⋅Lxxₘ⎠⋅M₄    ⎥
⎢                                                           ⎥
⎢                 1/2⋅Lxuₘ + Lxxₘ⋅M₄ + Lxu₂                 ⎥
⎢                                                           ⎥
⎢    ⎛             T     ⎞   ⎛             T     ⎞          ⎥
⎣1/2⋅⎝2⋅Luuₘ + 4⋅M₄ ⋅Lxuₘ⎠ + ⎝2⋅Luxₘ + 4⋅M₄ ⋅Lxxₘ⎠⋅M₄ + Luu₂⎦


In [459]:
L_G = block_collapse(L_Grad*E)
pprint(L_G)

[Lxₘ⋅M₁ + Lx₁  1/2⋅Luₘ + Lxₘ⋅M₂ + Lu₁  1/4⋅Lxₘ + Lx₂  1/2⋅Luₘ + Lxₘ⋅M₄ + Lu₂]


In [460]:
Syy = MatrixSymbol("Syy",n,n)
Syv = MatrixSymbol("Syy",n,m)
Svy = MatrixSymbol("Syy",m,n)
Svv = MatrixSymbol("Syy",m,m)
Sy = MatrixSymbol("Sy",1,n)
Sv = MatrixSymbol("Sv",1,m)
x = MatrixSymbol("x",n,1)
u = MatrixSymbol("u",m,1)
v = MatrixSymbol("v",m,1)

In [470]:
y = A*x+B*u+C*v
UV = BlockMatrix([[y.T,v.T]]).T
S = BlockMatrix([[Syy,Syv],[Svy,Svv]])
s = BlockMatrix([[Sy,Sv]])

P = block_collapse(UV.T*S*UV + s*UV)
pprint(P)

⎛⎛ T  T    T  T    T  T⎞        T    ⎞     ⎛⎛ T  T    T  T    T  T⎞        T  
⎝⎝u ⋅B  + v ⋅C  + x ⋅A ⎠⋅Syy + v ⋅Syy⎠⋅v + ⎝⎝u ⋅B  + v ⋅C  + x ⋅A ⎠⋅Syy + v ⋅S

  ⎞                                                
yy⎠⋅(A⋅x + B⋅u + C⋅v) + Sv⋅v + Sy⋅(A⋅x + B⋅u + C⋅v)


In [423]:
Syy,Syv,Svy,Svv,Sy,Sv = symbols("Syy Syv Svy Svv Sy Sv")
x,u,v = symbols("x u v")
A,B,C = symbols("A B C")

y = A*x + B*u + C*v
UV = Matrix([[y,v]]).T
S = Matrix([[Syy,Syv],[Svy,Svv]])
s = Matrix([[Sy,Sv]])
P = UV.T*S*UV #+ s*UV
P = P[0]

pprint(P.expand().factor(x,u,v))

 2  ⎛ 2    ⎞                                                       ⎛ 2      2 
x ⋅u⎝A ⋅Syy⎠ + x⋅u(2⋅A⋅B⋅Syy⋅u + v⋅(2⋅A⋅C⋅Syy + A⋅Svy + A⋅Syv)) + u⎝B ⋅Syy⋅u  

                                     2 ⎛ 2                          ⎞⎞
+ u⋅v⋅(2⋅B⋅C⋅Syy + B⋅Svy + B⋅Syv) + v ⋅⎝C ⋅Syy + C⋅Svy + C⋅Syv + Svv⎠⎠


In [437]:
P1 = S*UV
pprint(P1.expand())

⎡A⋅Syy⋅x + B⋅Syy⋅u + C⋅Syy⋅v + Syv⋅v⎤
⎢                                   ⎥
⎣A⋅Svy⋅x + B⋅Svy⋅u + C⋅Svy⋅v + Svv⋅v⎦


In [446]:
P2 = UV.T*P1
pprint(P2[0].expand().factor(x,u,v))

 2      2                    2      2                                      2 ⎛
A ⋅Syy⋅x  + 2⋅A⋅B⋅Syy⋅u⋅x + B ⋅Syy⋅u  + u⋅v⋅(2⋅B⋅C⋅Syy + B⋅Svy + B⋅Syv) + v ⋅⎝

 2                          ⎞                                  
C ⋅Syy + C⋅Svy + C⋅Syv + Svv⎠ + v⋅x⋅(2⋅A⋅C⋅Syy + A⋅Svy + A⋅Syv)
